# Geocoding slcs, etc. 

This notebook uses uavsar_pytools to geocode the uavsar data. 

In [2]:
from uavsar_pytools.georeference import geolocate_uavsar
from pathlib import Path
import sys
import os

scripts_dir = (os.path.dirname(os.getcwd()) + '/scripts/')
sys.path.append(scripts_dir)

from plotting import plot_tifs_grid 

/Users/julo9057/anaconda3/envs/geog6655_coherence/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(os.getcwd())
slc_dir = "/Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/slcs/"
data_dir = "/Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/"
in_dir = Path(slc_dir)
slcs_05208 = sorted(list(in_dir.glob('*05208*.slc')))
anns_05208 = sorted(list(in_dir.glob('*05208*.ann')))
slcs_23205 = sorted(list(in_dir.glob('*23205*.slc')))
anns_23205 = sorted(list(in_dir.glob('*23205*.ann')))
llh_05208 = slc_dir + "lowman_05208_01_BU_s2_2x8.llh"
llh_23205 = slc_dir + "lowman_23205_01_BC_s2_2x8.llh"
print(slcs_05208)
print(anns_05208)

/Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/notebooks
[PosixPath('/Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/slcs/lowman_05208_20002_006_200131_L090VV_01_BU_s2_2x8.slc'), PosixPath('/Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/slcs/lowman_05208_20007_002_200213_L090VV_01_BU_s2_2x8.slc'), PosixPath('/Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/slcs/lowman_05208_20011_002_200221_L090VV_01_BU_s2_2x8.slc'), PosixPath('/Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/slcs/lowman_05208_20016_003_200311_L090VV_01_BU_s2_2x8.slc'), PosixPath('/Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/slcs/lowman_05208_21009_005_210203_L090VV_01_BU_s2_2x8.slc'), 

In [23]:
print(f"Data dir: {data_dir}")
print(f"Lat/Lon file: {llh_05208}")
print(f"SLC file: {slcs_05208[0]}")
print(f"Annotation file: {anns_05208[0]}")

geolocate_uavsar(str(slcs_05208[0]), str(anns_05208[0]), data_dir, llh_05208)

Data dir: /Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/
Lat/Lon file: /Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/slcs/lowman_05208_01_BU_s2_2x8.llh
SLC file: /Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/slcs/lowman_05208_20002_006_200131_L090VV_01_BU_s2_2x8.slc
Annotation file: /Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/slcs/lowman_05208_20002_006_200131_L090VV_01_BU.ann


/Users/julo9057/anaconda3/envs/geog6655_coherence/lib/python3.13/site-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/julo9057/anaconda3/envs/geog6655_coherence/lib/python3.13/site-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(


['/Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/lowman_05208_20002_006_200131_L090VV_01_BU_s2_2x8.slc.real.tif',
 '/Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/lowman_05208_20002_006_200131_L090VV_01_BU_s2_2x8.slc.imag.tif']

In [1]:
img_list = ['/Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/lowman_05208_20002_006_200131_L090VV_01_BU_s2_2x8.slc.real.tif',
 '/Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/geog6655_coherence/data/data/snowex_lowman/lowman_05208_20002_006_200131_L090VV_01_BU_s2_2x8.slc.imag.tif']

Here are loops to run the geocoding script for each slc. It ran for about 35 minutes for all 23 SLCs

In [4]:
for slc, ann in zip(slcs_23205, anns_23205):
    geolocate_uavsar(str(slc), str(ann), data_dir, llh_23205)

for slc, ann in zip(slcs_05208, anns_05208):
    geolocate_uavsar(str(slc), str(ann), data_dir, llh_05208)

/Users/julo9057/anaconda3/envs/geog6655_coherence/lib/python3.13/site-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/julo9057/anaconda3/envs/geog6655_coherence/lib/python3.13/site-packages/osgeo/gdal.py:330: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
/Users/julo9057/anaconda3/envs/geog6655_coherence/lib/python3.13/site-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/julo9057/anaconda3/envs/geog6655_coherence/lib/python3.13/site-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/julo9057/anaconda3/envs/geog6